# Выбор локации для скважины

Как всегда, прежде всего загружаю библиотеки.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import plot_confusion_matrix

На всякий случай гружу всё. Теперь знакомлюсь с файлами данных по георазведке.

## Загрузка и подготовка данных

In [ ]:
exploration_1 = pd.read_csv('/datasets/geo_data_0.csv')
exploration_2 = pd.read_csv('/datasets/geo_data_1.csv')
exploration_3 = pd.read_csv('/datasets/geo_data_2.csv')

Далее проверяю содержимое файлов. Смотрю несколько строк файла, типы данных, количество пропусков, дубликатов.

In [ ]:
exploration_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [ ]:
exploration_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
exploration_1.duplicated().sum()

0

С первым файлом данных всё в порядке: пропусков нет, дубликатов нет, типы данных отторжения не вызывают. Аналогично проверяю два других файла.

In [ ]:
exploration_2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [ ]:
exploration_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
exploration_2.duplicated().sum()

0

In [ ]:
exploration_3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [ ]:
exploration_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
exploration_3.duplicated().sum()

0

In [ ]:
exploration_1 = exploration_1.rename(columns={'f0': 'f0_1', 'f1': 'f1_1', 'f2': 'f2_1', 'product': 'product_1'})
exploration_2 = exploration_2.rename(columns={'f0': 'f0_2', 'f1': 'f1_2', 'f2': 'f2_2', 'product': 'product_2'})
exploration_3 = exploration_3.rename(columns={'f0': 'f0_3', 'f1': 'f1_3', 'f2': 'f2_3', 'product': 'product_3'})

Второй и третий файлы также вопросов не вызывают пока что.

In [ ]:
exploration_1 = exploration_1.drop('id', axis=1)
exploration_2 = exploration_2.drop('id', axis=1)
exploration_3 = exploration_3.drop('id', axis=1)

In [ ]:
exploration_1.head()

,f0_1,f1_1,f2_1,product_1
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


### Выводы по разделу 1

1. Файлы данных загружены;
2. Размеры файлов одинаковы;
3. Пропусков и дубликатов не обнаружено;
4. Типы данных адекватны, в корректировке не нуждаются.

## Обучение и проверка модели

Для обучения и проверки модели необходимо выполнить несколько этапов.

### Разбиение данных на обучающую и валидационную выборки в соотношении 75:25

#### Первый регион

Разбиваю выборки в рекомендованных пропорциях.

In [ ]:
exploration_1_train, exploration_1_valid = train_test_split(exploration_1, test_size=0.25, random_state=12345)

In [ ]:
exploration_1_train.shape

(75000, 4)

In [ ]:
exploration_1_valid.shape

(25000, 4)

Создаю переменные для признаков и целевого признака.

In [ ]:
features_train_1 = exploration_1_train.drop('product_1', axis=1)
target_train_1 = exploration_1_train['product_1']

In [ ]:
features_valid_1 = exploration_1_valid.drop('product_1', axis=1)
target_valid_1 = exploration_1_valid['product_1']

Масштабирую выборки и проверяю, что получилось.

In [ ]:
scaled_columns_1 = ['f0_1', 'f1_1', 'f2_1']

scaler = StandardScaler()
scaler.fit(features_train_1[scaled_columns_1])

StandardScaler()

In [ ]:
features_train_1[scaled_columns_1] = scaler.transform(features_train_1[scaled_columns_1])
features_train_1.head()

,f0_1,f1_1,f2_1
27212,-0.544828,1.390264,-0.094959
7866,1.455912,-0.480422,1.209567
62041,0.260460,0.825069,-0.204865
70185,-1.837105,0.010321,-0.147634
82230,-1.299243,0.987558,1.273181


In [ ]:
features_valid_1[scaled_columns_1] = scaler.transform(features_valid_1[scaled_columns_1])
features_valid_1.head()

,f0_1,f1_1,f2_1
71751,0.517917,-0.610097,-0.126226
80493,0.568391,-0.086063,-0.814914
2655,0.805688,-1.613289,-1.341342
53233,0.222503,-1.355437,-0.597275
91141,-0.087941,1.433113,-0.544588


Обучаю модель, нахожу среднеквдратическую ошибку.

In [ ]:
model_1 = LinearRegression()
model_1.fit(features_valid_1, target_valid_1)
predictions_valid_1 = model_1.predict(features_valid_1)

In [ ]:
rmse_1 = mean_squared_error(target_valid_1, predictions_valid_1) ** .5
rmse_1

37.570336161076646

In [ ]:
reserves_1 = predictions_valid_1.mean()
reserves_1

92.07859674082927

Итого, средние запасы по первому региону 92,07 тыс баррелей,  а ошибка - 37,57.

#### Второй регион

Действую аналогичным предыдущему пункту образом.

In [ ]:
exploration_2_train, exploration_2_valid = train_test_split(exploration_2, test_size=0.25, random_state=12345)

In [ ]:
exploration_2_train.shape

(75000, 4)

In [ ]:
exploration_2_valid.shape

(25000, 4)

Создаю переменные для признаков и целевого признака.

In [ ]:
features_train_2 = exploration_2_train.drop('product_2', axis=1)
target_train_2 = exploration_2_train['product_2']

In [ ]:
features_valid_2 = exploration_2_valid.drop('product_2', axis=1)
target_valid_2 = exploration_2_valid['product_2']

Масштабирую выборки.

In [ ]:
scaled_columns_2 = ['f0_2', 'f1_2', 'f2_2']

scaler = StandardScaler()
scaler.fit(features_train_2[scaled_columns_2])

StandardScaler()

In [ ]:
features_train_2[scaled_columns_2] = scaler.transform(features_train_2[scaled_columns_2])
features_train_2.head()

,f0_2,f1_2,f2_2
27212,-0.850855,0.624428,0.296943
7866,1.971935,1.832275,0.294333
62041,1.079305,0.170127,-0.296418
70185,-1.512028,-0.887837,-0.880471
82230,-1.804775,-0.718311,-0.293255


In [ ]:
features_valid_2[scaled_columns_2] = scaler.transform(features_valid_2[scaled_columns_2])
features_valid_2.head()

,f0_2,f1_2,f2_2
71751,-0.168616,0.571372,0.297471
80493,0.878384,-1.772903,-0.293901
2655,-0.852976,-0.004986,-0.876502
53233,1.496940,-1.146077,-0.294011
91141,0.552177,0.057961,-1.457764


Обучаю модель и оцениваю средние запасы.

In [ ]:
model_2 = LinearRegression()
model_2.fit(features_valid_2, target_valid_2)
predictions_valid_2 = model_2.predict(features_valid_2)

In [ ]:
rmse_2 = mean_squared_error(target_valid_2, predictions_valid_2) ** .5
rmse_2

0.8930288727960609

In [ ]:
reserves_2 = predictions_valid_2.mean()
reserves_2

68.72313602435997

Итого, средние запасы по второму региону 68,72 тыс баррелей,  а ошибка - 0,89.

#### Третий регион

Разбиваю выборки.

In [ ]:
exploration_3_train, exploration_3_valid = train_test_split(exploration_3, test_size=0.25, random_state=12345)

In [ ]:
exploration_3_train.shape

(75000, 4)

In [ ]:
exploration_2_valid.shape

(25000, 4)

Создаю переменные для признаков и целевого признака.

In [ ]:
features_train_3 = exploration_3_train.drop('product_3', axis=1)
target_train_3 = exploration_3_train['product_3']

In [ ]:
features_valid_3 = exploration_3_valid.drop('product_3', axis=1)
target_valid_3 = exploration_3_valid['product_3']

Масштабирую выборки.

In [ ]:
scaled_columns_3 = ['f0_3', 'f1_3', 'f2_3']

scaler = StandardScaler()
scaler.fit(features_train_3[scaled_columns_3])

StandardScaler()

In [ ]:
features_train_3[scaled_columns_3] = scaler.transform(features_train_3[scaled_columns_3])
features_train_3.head()

,f0_3,f1_3,f2_3
27212,-0.526160,0.776329,-0.400793
7866,-0.889625,-0.404070,-1.222936
62041,-1.133984,0.208576,0.296765
70185,1.227045,1.570166,-0.764556
82230,-0.194289,0.878312,0.840821


In [ ]:
features_valid_3[scaled_columns_3] = scaler.transform(features_valid_3[scaled_columns_3])
features_valid_3.head()

,f0_3,f1_3,f2_3
71751,-0.836717,-2.227439,-0.078489
80493,-0.821648,0.740237,-1.000811
2655,-2.651336,-0.235721,-0.244546
53233,1.077998,0.938090,0.511313
91141,-1.173938,2.387270,1.034363


Обучаю модель, считаю средние запасы и ошибку.

In [ ]:
model_3 = LinearRegression()
model_3.fit(features_valid_3, target_valid_3)
predictions_valid_3 = model_3.predict(features_valid_3)

In [ ]:
rmse_3 = mean_squared_error(target_valid_3, predictions_valid_3) ** .5
rmse_3

40.02297245848588

In [ ]:
reserves_3 = predictions_valid_3.mean()
reserves_3

94.88423280885435

Итого, средние запасы по второму региону 94,88 тыс баррелей,  а ошибка - 40,02.

Все полученные данные сведу в табличку, чтобы при необходимости было удобно обратиться к полученным результатам.

In [ ]:
regions = ['Регион 1', 'Регион 2', 'Регион 3']
reserves = [reserves_1, reserves_2, reserves_3]
error = [rmse_1, rmse_2, rmse_3]


reserves_data = pd.DataFrame({'Регион': regions,
                           'Средние запасы': reserves,
                           'Отклонение': error })
reserves_data

,Регион,Средние запасы,Отклонение
0,Регион 1,92.078597,37.570336
1,Регион 2,68.723136,0.893029
2,Регион 3,94.884233,40.022972


#### Выводы по разделу 2
1. Для каждого региона проведено разбиение данных на обучающую и валидационную выборки в пропорции 75%/25%;
2. Для каждого региона проведено обучение модели на валидационной выборке;
3. Предсказаны средние запасы по каждому региону:

   3.1 Наибольшими запасами обладает третий регион;
   
   3.2 Наименьшей ошибкой в предсказании обладает второй регион.

## Подготовка к расчёту прибыли

Все ключевые значения для расчётов сохраняю в отдельных переменных.

In [ ]:
target_valid_1 = pd.Series(target_valid_1).reset_index(drop=True)
predictions_valid_1 = pd.Series(predictions_valid_1).reset_index(drop=True)

target_valid_2 = pd.Series(target_valid_2).reset_index(drop=True)
predictions_valid_2 = pd.Series(predictions_valid_2).reset_index(drop=True)

target_valid_3 = pd.Series(target_valid_3).reset_index(drop=True)
predictions_valid_3 = pd.Series(predictions_valid_3).reset_index(drop=True)

Необходимые для расчёта константы сохраняю отдельно. Обозначу их прописными буквами.

In [ ]:
EXPLORATION_SPOTS = 500 # Число точек, исследуемых при разведке региона
BEST_SPOTS = 200 # Количество лучших точек, выбранных по результатам разведки региона
WELLS_BUDGET = 10000000000 # Бюджет на разработку скважин в регионе (руб)
BARREL_PRICE = 450 # Стоимость барреля нефти (руб)
UNIT_PRICE = 450000 # Стоимость единицы продукта (тысячи баррелей) (руб)
SAMPLES=1000 # Количество выборок для нахождения прибыли

Вычислю минимальный рентабельный объём запасов

In [ ]:
raw_min_volume = (WELLS_BUDGET / BEST_SPOTS)/UNIT_PRICE
print ('Минимальные рентабельные запасы на скважину =', raw_min_volume, 'тыс. баррелей' )

Минимальные рентабельные запасы на скважину = 111.11111111111111 тыс. баррелей


Вспомним нашу таблицу из предыдущего пункта. Расчёт минимального рентабельного дебита показывает, что в каждом регионе далеко не все скважины обладают необходимым объёмом запасов. Поэтому посмотрю, сколько скважин по регионам соответствует полученному пороговому объёму запасов.

In [ ]:
print('Процент скважин с рентабельным объёмом запасов в 1 регионе =',
      (len(exploration_1.query("product_1 > @raw_min_volume"))/len(exploration_1))*100, '%')
print('Процент скважин с рентабельным объёмом запасов в 2 регионе =',
      (len(exploration_2.query("product_2 > @raw_min_volume"))/len(exploration_2))*100, '%')
print('Процент скважин с рентабельным объёмом запасов в 3 регионе =',
      (len(exploration_3.query("product_3 > @raw_min_volume"))/len(exploration_3))*100, '%')
print('Количество скважин с рентабельным объёмом в 1 регионе =',
      len(exploration_1.query("product_1 > @raw_min_volume")), 'штук')
print('Количество скважин с рентабельным объёмом во 2 регионе =',
      len(exploration_2.query("product_2 > @raw_min_volume")), 'штук')
print('Количество скважин с рентабельным объёмом в 3 регионе =',
      len(exploration_3.query("product_3 > @raw_min_volume")), 'штук')

Процент скважин с рентабельным объёмом запасов в 1 регионе = 36.583 %
Процент скважин с рентабельным объёмом запасов в 2 регионе = 16.537 %
Процент скважин с рентабельным объёмом запасов в 3 регионе = 38.178 %
Количество скважин с рентабельным объёмом в 1 регионе = 36583 штук
Количество скважин с рентабельным объёмом во 2 регионе = 16537 штук
Количество скважин с рентабельным объёмом в 3 регионе = 38178 штук


### Выводы по разделу 3
1. Подготовлены данные для расчёта прибыли;
2. По каждому региону определены процент и количество скважин с рентабельным объёмом запасов;
3. Наибольшее количество рентабельных скважин в третьем регионе, наименьшее - во втором.

## Расчёт прибыли и рисков

### Написание функции для расчёта прибыли по выбранным скважинам и предсказаниям модели

Итак, нужная нам функция принимает целевые значения объёмов запасов скважин, предсказания модели по запасам и лучшие точки при разведке.
Выдаёт прибыль для полученного объёма.

In [ ]:
def profit (target, predictions, BEST_SPOTS):
    predictions_sorted = predictions.sort_values(ascending=False)
    selected = target[predictions_sorted.index][:BEST_SPOTS]
    return UNIT_PRICE * selected.sum() - WELLS_BUDGET

Проверю на 1 регионе.

In [ ]:
profit(target_valid_1, predictions_valid_1, BEST_SPOTS)

3316629340.6640186

### Нахождение средней прибыли, 95%-ого доверительного интервала и риска убытков.

Напишк цикл, чтобы учитывались все регионы.

In [ ]:
target_total = [target_valid_1, target_valid_2, target_valid_3]
prediction_total = [predictions_valid_1, predictions_valid_2, predictions_valid_3]
regions_total = ['Регион 1', 'Регион 2', 'Регион 3']

state = np.random.RandomState(12345)

for k in range(len(target_total)):
    values = []
    for i in range(SAMPLES):
        indiv_target = target_total[k].sample(n=EXPLORATION_SPOTS , replace=True, random_state=state)
        indiv_predictions = prediction_total[k].loc[indiv_target.index]
        values.append(profit(indiv_target.reset_index(drop=True), indiv_predictions.reset_index(drop=True), BEST_SPOTS))

    values = pd.Series(values)
    lower = values.quantile(.025)
    upper = values.quantile(.975)

    mean = values.mean()

    loss_probability = sum(values < 0) / values.count() * 100

    print(regions[k])
    print('Вероятная прибыль:', mean)
    print('95%-доверительный интервал:', 'от', lower, 'до', upper)
    print('Вероятность убытков:', loss_probability, '%')

Регион 1
Вероятная прибыль: 395809611.7697294
95%-доверительный интервал: от -119981223.3770875 до 911418854.9313884
Вероятность убытков: 6.5 %
Регион 2
Вероятная прибыль: 461165831.4503767
95%-доверительный интервал: от 78050810.75173984 до 862952060.2637234
Вероятность убытков: 0.7000000000000001 %
Регион 3
Вероятная прибыль: 395960984.0533397
95%-доверительный интервал: от -101505758.28928727 до 940808059.3553646
Вероятность убытков: 5.6000000000000005 %


### Выводы по разделу 4
1. Написана функция для расчёта прибыли.
2. Получены результаты расётов прибыли и рисков для каждого региона.
3. Несмотря на меньший процент рентабельных скважин наибольшей вероятной прибылью обладает Регион 2, этот же регион обладает наименьшей вероятностью убытков.

## Общие выводы

1. Загружены и подготовлены данные по трём регионам разведки;
2. Для каждого региона проведено разбиение данных на обучающую и валидационную выборки в пропорции 75%/25%;
3. Для каждого региона проведено обучение модели на валидационной выборке;
4. Предсказаны средние запасы по каждому региону:

   4.1 Наибольшими запасами обладает третий регион;
   
   4.2 Наименьшей ошибкой в предсказании обладает второй регион;
5. Подготовлены данные для расчёта прибыли;
6. По каждому региону определены процент и количество скважин с рентабельным объёмом запасов;
7. Наибольшее количество рентабельных скважин в третьем регионе, наименьшее - во втором;
8. Написана функция для расчёта прибыли.
9. Получены результаты расётов прибыли и рисков для каждого региона.
10. Несмотря на меньший процент рентабельных скважин наибольшей вероятной прибылью обладает Регион 2, этот же регион обладает наименьшей вероятностью убытков.
11. Таким образом, наиболее подходящим для разработки является второй регион.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован